### 买股票问题 I-IV
```
今天leetcode的每日一题是这个系列的第二题，顺便把其他的也做了
```

#### 121. Best Time to Buy and Sell Stock 
```
Say you have an array for which the ith element is the price of a given stock on day i.

If you were only permitted to complete at most one transaction (i.e., buy one and sell one share of the stock), design an algorithm to find the maximum profit.

Note that you cannot sell a stock before you buy one.

来源：力扣（LeetCode）
链接：https://leetcode-cn.com/problems/best-time-to-buy-and-sell-stock
著作权归领扣网络所有。商业转载请联系官方授权，非商业转载请注明出处。
```

- 题解1：一系列问题的第一个，只能购买一次，求最大的买法。类似的问题实质上是求最大差值，需要分别知道前一部分的最小值和后一部分的最大值（因为本题背景是后减前）。最常用的方法是扫描/遍历法，即先从前往后求出到第i个为止前面的最小项，然后从后往前扫描，求出到第i个为止可能出现的最大差值(vals[i] - mins[i-1]),再从这些差值中取最大值。
```python
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        # N**2 out of time limit
        # Time complexity: 2*N
        if not prices: return 0

        max_val = 0
        min_val = float("inf")
        
        for i in range(len(prices)):
            p = prices[i]
            if p < min_val: min_val = p
            else:
                max_val = p - min_val if p - min_val > max_val else max_val

        return max_val
```  

#### 122. Best Time to Buy and Sell Stock II

- 题解1：系列中第二个，引入DP的起始问题，与原始问题的区别在于不限transaction次数。解决关键是如何想出DP表达式以及状态转移方程。
    - 假设DP[i][j]表示到达第i次时可能的最大利润，j为binary，分别代表“手持股票”与“当下没有股票”两种状态。
    - 由此可得，DP[i][0] = max(DP[i-1][0],DP[i-1][1] + prices[i]) # 之前就没有或者之前有而本轮抛售
    - DP[i][1] = max(DP[i-1][1],DP[i-1][0] - prices[i])
    - 最终结果： DP[-1][0] (把股票卖完肯定比手里还拿着强)
    - * DP问题结束后有条件的情况一定要考虑能不能**压缩状态**，本题中每一次DP的状态转移都只取决于前一次的DP，因此状态压缩可行。下面的是原始版本，以后有时间可以自己改一下。
```python3
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        if not prices: return 0

        dp = [[0,0] for _ in range(len(prices))]

        dp[0][0] = 0
        dp[0][1] = -prices[0]

        for i in range(1,len(prices)):
            dp[i][0] = max(dp[i-1][0],dp[i-1][1] + prices[i])
            dp[i][1] = max(dp[i-1][0]-prices[i],dp[i-1][1])
        
        return dp[-1][0]
```

- 题解2：因为本体不限购买次数，加上需要计算最大和，如果最大和 = 每个局部最大值之和，那么greedy approch就可以被应用。本题符合此条件（证明参见官方解法，也可以举两个例子想一想）。换句话说可以直接遍历所有可能的差值，只要差值大于零就进行采纳，最终和必定为全局最大和。
```python
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        if not prices: return 0

        # Greedy Solution
        ans = 0

        for i in range(1,len(prices)):
            ans += max(0,prices[i] - prices[i-1])
        
        return ans
```

#### 123. Best Time to Buy and Sell Stock III

- 题解：前一题的进阶版，区别在于限制最多进行2次转账
    - 由于最多只能进行两次转账，原有的DP状态转移方程不足包含所有限制条件，因此引入一个新的数组[k]代表之前购买了多少次，新状态转移方程的思路和之前相似。
```python3
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        # 使用状态压缩 -- 做DP问题最后不要忘了关注这点！！！
        if not prices: return 0

        dp = [[None,None,None] for _ in range(2)]
        dp[0][0],dp[0][1],dp[1][0],dp[1][1] = 0,-float("inf"),-prices[0],-float("inf")
        dp[0][2],dp[1][2] = -float("inf"),-float("inf")

        for i in range(1,len(prices)):
            temp  = [dp[0][0],max(dp[0][1],dp[1][0] + prices[i]),max(dp[0][0] - prices[i], dp[1][0]),max(dp[0][1] - prices[i], dp[1][1]),max(dp[0][2],dp[1][1]+prices[i]),-float("inf")]
            dp[0][0],dp[0][1],dp[1][0],dp[1][1],dp[0][2],dp[1][2] = temp

        return max(dp[0])
```

#### 188. Best Time to Buy and Sell Stock IV  

- 题解：再进阶版，区别在于限制最多进行k次转账
    - 解决思路类似于那个Linked list的系列问题，能直接等效成k说明中间的操作存在通式，找到这个递推通式再处理一下头尾就完了，过程也有点像压缩DP解法的状态。
    - 一开始做错的原因是计算第K次卖出时的状态转移方程出错，如果是卖出K的数值是不需要改变的。
```python3
class Solution:
    def maxProfit(self, k: int, prices: List[int]) -> int:
        if not prices: return 0
        if not k: return 0
        # 状态压缩版本

        dp = [[None for j in range(k+1)] for _ in range(2)]
        
        dp[0][0],dp[1][0] = 0,-prices[0]
        for j in range(1,k+1):
            dp[0][j] = -float("inf")
            dp[1][j] = -float("inf")

        temp = None
        for i in range(1,len(prices)):
            temp = []
            temp.extend([dp[0][0],max(dp[1][0],dp[0][0] - prices[i])])
            for j in range(1,k):
                temp.append(max(dp[0][j],dp[1][j-1] + prices[i]))
                temp.append(max(dp[1][j],dp[0][j] - prices[i]))
                # 错误原因：上面一行 -- 如果是卖出的情况买卖次数是不需要改动的。
            temp.append(max(dp[0][k],dp[1][k-1] + prices[i]))
            temp.append(-float("inf"))
            index = 0
            for q in range(k+1):
                dp[0][q],index = temp[index],index + 1
                dp[1][q],index = temp[index],index + 1

        return max(dp[0])```
        